In [1]:
import os
from pathlib import Path
import sys
import os
import _pickle as pickle
import json
import copy 
import pandas
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.backends.backend_pdf import PdfPages
import csv
import json
from collections import Counter
import statistics 
import plotly.graph_objects as go
import math


In [2]:
from itertools import combinations
from scipy.stats import ttest_ind
import decimal

# create a new context for this task
ctx = decimal.Context()

# 20 digits should be enough for everyone :D
ctx.prec = 20

def float_to_str(f):
    """
    Convert the given float to a string,
    without resorting to scientific notation
    """
    d1 = ctx.create_decimal(repr(f))
    return format(d1, 'f')

In [3]:
with open('../../responseJudgement.json') as json_file:
    judgements = json.load(json_file)
    
with open('../../responseJudgement_sugg.json') as json_file:
    suggJudgements = json.load(json_file)

In [4]:
len(judgements)

15417

In [5]:
len(suggJudgements)

1014

In [6]:
cleanPath = '../../clean/'
settingName = 'Setting 5'

In [7]:
responseSet = {}

for root, dirs, files in os.walk(cleanPath):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    currFileName = data['currentFileName']
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        uniqueStr = 'fileIndex=' + str(context['fileIndex']) + '&&contextLen=' + str(context['contextLen']) + '&&userResponse=' + context['userResponse'].strip()
                        
                        if(uniqueStr in responseSet):
                            continue
                        
                        else:
                            
                            foundMatches = [x for x in judgements if x['Input.contextLen'] == str(context['contextLen']) and x['Input.fileIndex'] == str(context['fileIndex'])]
                            foundJudgement = [x for x in foundMatches if x['Input.fileName'] == currFileName and x['Input.localIndex'] == str(context['localIndex'])]
                            
                            if(len(foundJudgement) > 0):
                                responseSet[uniqueStr] = foundJudgement
                        
                except Exception as e: print(e)

In [8]:
len(responseSet)

5139

In [9]:
suggSet = {}

for root, dirs, files in os.walk(cleanPath):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    currFileName = data['currentFileName']
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        suggestions = context['suggestions']
                        
                        for sugg in suggestions:
                            uniqueStr = 'fileIndex=' + str(context['fileIndex']) + '&&contextLen=' + str(context['contextLen']) + '&&userResponse=' + sugg.strip()

                            if(uniqueStr in responseSet or uniqueStr in suggSet):
                                continue

                            else:

                                foundMatches = [x for x in suggJudgements if x['Input.contextLen'] == str(context['contextLen']) and x['Input.fileIndex'] == str(context['fileIndex'])]
                                foundJudgement = [x for x in foundMatches if x['Input.fileName'] == currFileName and x['Input.localIndex'] == str(context['localIndex'])]

                                if(len(foundJudgement) > 0):
                                    suggSet[uniqueStr] = foundJudgement
                        
                except Exception as e: print(e)

In [10]:
len(suggSet)

458

In [11]:
Qualtype = 'Correctness'
metric = {
    'Answer.' + Qualtype +'.Strongly Disagree': 1,
    'Answer.' + Qualtype +'.Disagree': 2,
    'Answer.' + Qualtype +'.Neutral': 3,
    'Answer.' + Qualtype +'.Agree': 4,
    'Answer.' + Qualtype +'.Strongly Agree': 5
}

In [12]:
scoresHuman = []

for root, dirs, files in os.walk(cleanPath + 'Setting 1'):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        
                        userResponse = context['userResponse']
                        fileIndex = context['fileIndex']
                        contextLen = context['contextLen']
                        
                        identifierStr = 'fileIndex=' + str(fileIndex) + '&&contextLen=' + str(contextLen) + '&&userResponse=' + userResponse.strip()
                        
                        responseJudgements =  responseSet[identifierStr]
                        
                        currFeedback = []
                        for feedback in responseJudgements:
                            for key in feedback:
                                
                                if('Answer.' + Qualtype in key and feedback[key] == 'true'):
                                    currFeedback.append(metric[key])
                        
                        scoresHuman.append(statistics.mean(currFeedback))
                            
                        
                    
                except Exception as e: print(e)                 

In [13]:
scoresBot = []

for root, dirs, files in os.walk(cleanPath + settingName):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        
                        userResponse = context['userResponse']
                        fileIndex = context['fileIndex']
                        contextLen = context['contextLen']
                        suggestions = context['suggestions']
                        
                        identifierStr = 'fileIndex=' + str(fileIndex) + '&&contextLen=' + str(contextLen) + '&&userResponse=' + suggestions[0].strip()
                        
                        if(identifierStr in responseSet):
                            responseJudgements =  responseSet[identifierStr]
                        else:
                            responseJudgements =  suggSet[identifierStr]
                        
                        currFeedback = []
                        for feedback in responseJudgements:
                            for key in feedback:
                                
                                if('Answer.' + Qualtype in key and feedback[key] == 'true'):
                                    currFeedback.append(metric[key])
                        
                        scoresBot.append(statistics.mean(currFeedback))
                            
                        
                    
                except Exception as e: print(e)                 

In [14]:
sugg1 = []
sugg3 = []
sugg5 = []

autoSugg1 = []
autoSugg3 = []
autoSugg5 = []


for root, dirs, files in os.walk(cleanPath + settingName):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        
                        userResponse = context['userResponse']
                        fileIndex = context['fileIndex']
                        contextLen = context['contextLen']
                        
                        identifierStr = 'fileIndex=' + str(fileIndex) + '&&contextLen=' + str(contextLen) + '&&userResponse=' + userResponse.strip()
                        
                        responseJudgements =  responseSet[identifierStr]
                        
                        currFeedback = []
                        
                        for feedback in responseJudgements:
                            for key in feedback:
                                
                                if('Answer.' + Qualtype in key and feedback[key] == 'true'):
                                    currFeedback.append(metric[key])
                        
                        
                        if(data['autoFilledSuggestion'] == 0):
                            
                            if(data['suggestionsNumber'] == 1):
                                sugg1.append(statistics.mean(currFeedback))

                            elif(data['suggestionsNumber'] == 3):
                                sugg3.append(statistics.mean(currFeedback))

                            elif(data['suggestionsNumber'] == 5):
                                sugg5.append(statistics.mean(currFeedback))
                        
                        else:

                            if(data['suggestionsNumber'] == 1):
                                autoSugg1.append(statistics.mean(currFeedback))

                            elif(data['suggestionsNumber'] == 3):
                                autoSugg3.append(statistics.mean(currFeedback))
                            
                            elif(data['suggestionsNumber'] == 5):
                                autoSugg5.append(statistics.mean(currFeedback))
                    
                    
                except Exception as e: print(e)

In [15]:
sugg1Bot = []
sugg3Bot = []
sugg5Bot = []

autoSugg1Bot = []
autoSugg3Bot = []
autoSugg5Bot = []


for root, dirs, files in os.walk(cleanPath + settingName):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        
                        userResponse = context['userResponse']
                        fileIndex = context['fileIndex']
                        suggestions = context['suggestions']
                        
                        identifierStr = 'fileIndex=' + str(fileIndex) + '&&contextLen=' + str(contextLen) + '&&userResponse=' + suggestions[0].strip()
                        
                        if(identifierStr in responseSet):
                            responseJudgements =  responseSet[identifierStr]
                        else:
                            responseJudgements =  suggSet[identifierStr]
                        
                        currFeedback = []
                        
                        for feedback in responseJudgements:
                            for key in feedback:
                                
                                if('Answer.' + Qualtype in key and feedback[key] == 'true'):
                                    currFeedback.append(metric[key])
                        
                        
                        if(data['autoFilledSuggestion'] == 0):
                            
                            if(data['suggestionsNumber'] == 1):
                                sugg1Bot.append(statistics.mean(currFeedback))

                            elif(data['suggestionsNumber'] == 3):
                                sugg3Bot.append(statistics.mean(currFeedback))

                            elif(data['suggestionsNumber'] == 5):
                                sugg5Bot.append(statistics.mean(currFeedback))
                        
                        else:

                            if(data['suggestionsNumber'] == 1):
                                autoSugg1Bot.append(statistics.mean(currFeedback))

                            elif(data['suggestionsNumber'] == 3):
                                autoSugg3Bot.append(statistics.mean(currFeedback))
                            
                            elif(data['suggestionsNumber'] == 5):
                                autoSugg5Bot.append(statistics.mean(currFeedback))
                    
                    
                except Exception as e: print(e)

"fileIndex=1&&contextLen=1&&userResponse=No, I don't know, but I do know that the piano was invented in Italy by Bartholomew cristofori around 1700."
'fileIndex=9&&contextLen=1&&userResponse=Well, tacos are made with beef, pork, seafood, and seafood.'
"fileIndex=8&&contextLen=1&&userResponse=I'm not sure, but I know they're originally called 'klklklasse'. They're a German - style car."
'fileIndex=29&&contextLen=1&&userResponse=I have never been to the mall, but i have heard that it is a great place to get into.'
"fileIndex=28&&contextLen=1&&userResponse=I'm not sure but I know that they are typically built in hotels, hotels, and restaurants."
'fileIndex=11&&contextLen=1&&userResponse=I like to sew, it is the process of fastening objects with stitches using a needle and thread'
'fileIndex=18&&contextLen=1&&userResponse=Well, it is the science, technology, science, engineering, engineering, and engineering.'
'fileIndex=21&&contextLen=1&&userResponse=Well, it is a popular tourist destinat

In [16]:
settings = {
    'H': scoresHuman,
    'Bot': scoresBot,
    'sugg1': sugg1,
    'sugg3': sugg3,
    'sugg5': sugg5,
    'autoSugg1': autoSugg1,
    'autoSugg3': autoSugg3,
    'autoSugg5': autoSugg5
}

for list1, list2 in combinations(settings.keys(), 2):
    t, p = ttest_ind(settings[list1], settings[list2], equal_var = False)
    print (list1, list2, float_to_str(p))

H Bot 0.0000000000000000000000000000060546846645581
H sugg1 0.000012725072766164697
H sugg3 NaN
H sugg5 NaN
H autoSugg1 0.0000016375023413825638
H autoSugg3 NaN
H autoSugg5 NaN
Bot sugg1 0.00000000001973157582408558
Bot sugg3 NaN
Bot sugg5 NaN
Bot autoSugg1 0.000000000026468934127234222
Bot autoSugg3 NaN
Bot autoSugg5 NaN
sugg1 sugg3 NaN
sugg1 sugg5 NaN
sugg1 autoSugg1 0.7670295810247072
sugg1 autoSugg3 NaN
sugg1 autoSugg5 NaN
sugg3 sugg5 NaN
sugg3 autoSugg1 NaN
sugg3 autoSugg3 NaN
sugg3 autoSugg5 NaN
sugg5 autoSugg1 NaN
sugg5 autoSugg3 NaN
sugg5 autoSugg5 NaN
autoSugg1 autoSugg3 NaN
autoSugg1 autoSugg5 NaN
autoSugg3 autoSugg5 NaN


In [18]:
colors = ['#FAEBD7', '#FFA07A', '#B0C4DE', '#AFEEEE', '#8FBC8F']
x = ['1 Suggestion']
fig = go.Figure()

xHuman = ['']
yHuman = [scoresHuman]
yHumanErorr = [np.std(z)/math.sqrt(len(z)) for z in yHuman] 
yHuman = [round(statistics.mean(z), 2) for z in yHuman]

fig.add_trace(go.Bar(
    name='Human Only',
    x=xHuman, y=yHuman, text=yHuman, textposition='inside',insidetextanchor='middle',
    marker_color='#B0C4DE',
    marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.8,
    error_y=dict(type='data', array=yHumanErorr)
))

xBot = ['']
yBot = [scoresBot]
yBotErorr = [np.std(z)/math.sqrt(len(z)) for z in yBot] 
yBot = [round(statistics.mean(z), 2) for z in yBot]

fig.add_trace(go.Bar(
    name='Chatbot Only',
    x=xBot, y=yBot, text=yBot, textposition='inside',insidetextanchor='middle',
    marker_color='#A9A9A9',
    marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.8,
    error_y=dict(type='data', array=yBotErorr)
))


ySuggOnly = [sugg1]
ySuggOnlyErorr = [np.std(z)/math.sqrt(len(z)) for z in ySuggOnly] 
ySuggOnly = [round(statistics.mean(z), 2) for z in ySuggOnly]

fig.add_trace(go.Bar(
    name='C1',
    x=x, y=ySuggOnly, text=ySuggOnly, textposition='inside',insidetextanchor='middle',
    marker_color='lightsalmon',
    marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.8,
    error_y=dict(type='data', array=ySuggOnlyErorr)
))

ySuggOnlyBot = [sugg1Bot]
ySuggOnlyBotErorr = [np.std(z)/math.sqrt(len(z)) for z in ySuggOnlyBot] 
ySuggOnlyBot = [round(statistics.mean(z), 2) for z in ySuggOnlyBot]

fig.add_trace(go.Bar(
    name='C1 - Chatbot Only',
    x=x, y=ySuggOnlyBot, text=ySuggOnlyBot, textposition='inside',insidetextanchor='middle',
    marker_color='#ffdacc',
    marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.8,
    error_y=dict(type='data', array=ySuggOnlyBotErorr)
))


yAutoSugg = [autoSugg1]
yAutoSuggErorr = [np.std(z)/math.sqrt(len(z)) for z in yAutoSugg] 
yAutoSugg = [round(statistics.mean(z), 2) for z in yAutoSugg]

fig.add_trace(go.Bar(
    name='C2',
    x=x, y=yAutoSugg, text=yAutoSugg, textposition='inside',insidetextanchor='middle',
    marker_color='#AFEEEE',
    marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.8,
    error_y=dict(type='data', array=yAutoSuggErorr)
))


yAutoSuggBot = [autoSugg1Bot]
yAutoSuggBotErorr = [np.std(z)/math.sqrt(len(z)) for z in yAutoSuggBot] 
yAutoSuggBot = [round(statistics.mean(z), 2) for z in yAutoSuggBot]

fig.add_trace(go.Bar(
    name='C2 - Chatbot Only',
    x=x, y=yAutoSuggBot, text=yAutoSuggBot, textposition='inside',insidetextanchor='middle',
    marker_color='#eafbfb',
    marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.8,
    error_y=dict(type='data', array=yAutoSuggBotErorr)
))


fig.update_layout(
    font=dict(
        size=13
    )
)
fig.update_yaxes(range=[3, 4])

fig.update_layout(legend_orientation="h")
fig.show()